### ML and feature importance analysis to predict metabolite from proteomic data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import shap

#### read in data

In [ ]:
metab = pd.read_csv(r'C:\Users\momenzadeha\OneDrive - Cedars-Sinai Health System\yuming_drug_metabolite\meta_dupli.csv', index_col=0)
prot = pd.read_csv(r'C:\Users\momenzadeha\OneDrive - Cedars-Sinai Health System\yuming_drug_metabolite\pro_dupli.csv')

#### prepare metabolite and protein data

In [ ]:
metab = metab.loc[:, ~metab.columns.str.contains('DMSO|DFO', case=False)]
metab = metab.T
prot = prot.loc[:, ~prot.columns.str.contains('DMSO|DFO', case=False)]
prot.rename(columns = {'Unnamed: 0': 'Protein'}, inplace=True)
prot['Protein'] = prot['Protein'].str.split('|').str[-1]
prot=prot.set_index('Protein')
prot=prot.T

In [ ]:
#define model X (input) and y (output)
y = metab
X = prot
y = y.reindex(X.index)

In [ ]:
# Split into training and test sets
def train_test_split_by_drug(X, y, test_size=0.2, random_state=0):
    drug_base_names = X.index.str.extract(r'([a-zA-Z]+)')[0]
    gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_idx, test_idx in gss.split(X, groups=drug_base_names):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split_by_drug(X, y, test_size=0.2, random_state=0)

#### train random forest regressor to predict each metabolite

In [ ]:
# Initialize lists to store R² scores and metabolite names, and a dictionary for models
r2_scores = []
metabolite_names = y.columns
models = {}

# Loop through each metabolite, train a separate model, and store it in the dictionary
for metabolite in metabolite_names:
    # Set the target for the current metabolite
    y_train_metab = y_train[metabolite]
    y_test_metab = y_test[metabolite]
    
    # Initialize and train the model
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train_metab)
    
    # Make predictions and calculate the R² score
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test_metab, y_pred)
    r2_scores.append(r2)
    
    # Store the model in the dictionary
    models[metabolite] = model

# Create a DataFrame to store the R² scores for each metabolite
r2_df = pd.DataFrame({'Metabolite': metabolite_names, 'R2_Score': r2_scores})

# Plot a histogram of the R² scores
plt.figure(figsize=(10, 6))
plt.hist(r2_scores, bins=20, edgecolor='k')
plt.xlabel('R² Score')
plt.ylabel('Frequency')
plt.title('Distribution of R² Scores for Each Metabolite')
plt.show()

In [ ]:
r2_df[r2_df['R2_Score']>0.4]

In [ ]:
phenyl_model=models['phenylalanine']
spermidine_model=models['spermidine']

#### SHAP analysis

In [ ]:
def shap_analysis(model, X_test, feature_names):
    # Initialize the SHAP explainer
    explainer = shap.KernelExplainer(model.predict, X_test)
    # Calculate SHAP values for the test set
    shap_values = explainer.shap_values(X_test)
    shap_df = pd.DataFrame(shap_values, columns=feature_names)
    shap.summary_plot(shap_values, X_test, feature_names=feature_names, plot_type="bar")
    return shap_df

#beeswarm plot is used to show the distribution of SHAP values for all features across the entire test set
def shap_beeswarm_plot(model, X_test, output_path="beeswarm_plot.png"):
    # Check if X_test is a DataFrame
    if not isinstance(X_test, pd.DataFrame):
        raise ValueError("X_test should be a DataFrame with feature names as columns.")
    # Use TreeExplainer for efficiency with tree-based models
    explainer = shap.TreeExplainer(model)
    # Calculate SHAP values for all instances in the test set
    shap_values = explainer.shap_values(X_test)
    # Create the beeswarm plot
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values, X_test, plot_type="dot", show=False)
    # Customize the plot appearance (optional)
    plt.title("SHAP Beeswarm Plot")
    plt.xlabel("SHAP Value (Impact on Model Output)")
    # Save the plot to a file
    plt.savefig(output_path, format="png", bbox_inches="tight")
    plt.close()  # Close the plot to avoid display in notebooks

##### phenylalanine

In [ ]:
feature_names = X_test.columns  # Use the column names as feature names

In [ ]:
shap_df = shap_analysis(phenyl_model, X_test, feature_names)
shap_beeswarm_plot(phenyl_model, X_test)

##### spermidine

In [ ]:
shap_beeswarm_plot(spermidine_model, X_test)